# Build an Agent

* References: https://python.langchain.com/docs/tutorials/agents/

* Search Agent

## Setup

In [2]:
import os
import getpass

os.environ["LANGSMITH_TRACING"] = "true"

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API Key: ")
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key: ")

## Define Tools: Tavily search engine

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is the weather in Seoul?")
print(search_results)

## Define Tools: Web search
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [search]



[{'title': 'Weather in Seoul', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Seoul', 'region': '', 'country': 'South Korea', 'lat': 37.5664, 'lon': 126.9997, 'tz_id': 'Asia/Seoul', 'localtime_epoch': 1742281076, 'localtime': '2025-03-18 15:57'}, 'current': {'last_updated_epoch': 1742280300, 'last_updated': '2025-03-18 15:45', 'temp_c': 5.2, 'temp_f': 41.4, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 13.9, 'wind_kph': 22.3, 'wind_degree': 345, 'wind_dir': 'NNW', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.08, 'precip_in': 0.0, 'humidity': 45, 'cloud': 75, 'feelslike_c': 1.1, 'feelslike_f': 33.9, 'windchill_c': 1.0, 'windchill_f': 33.8, 'heatindex_c': 5.2, 'heatindex_f': 41.3, 'dewpoint_c': -4.5, 'dewpoint_f': 23.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 2.1, 'gust_mph': 17.5, 'gust_kph': 28.2}}", 'score': 0.9006186}, {'title': 'Weather Seoul in March 2

## Using Language Models

In [14]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="gpt-4o-mini", model_provider="openai")

from langchain.schema import HumanMessage
response = model.invoke([HumanMessage(content="Hi!")])
print(response.content)

model_with_tools = model.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="Hi")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


response = model_with_tools.invoke([HumanMessage(content="What is the weather in Seoul?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")



Hello! How can I assist you today?
ContentString: Hello! How can I assist you today?
ToolCalls: []
ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Seoul'}, 'id': 'call_Uy3Wh1RCzW6w9EtIgiEpCs2Q', 'type': 'tool_call'}]


## Create the agent

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [20]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Hi!")]})
response["messages"]

response = agent_executor.invoke({"messages": [HumanMessage(content="What is the weather in Seoul?")]})
response["messages"]


[HumanMessage(content='What is the weather in Seoul?', additional_kwargs={}, response_metadata={}, id='0d18c0e0-988d-4acf-9cb1-a909f0ed008f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HnDTh6kMR5HYZZzPRDfYMvBS', 'function': {'arguments': '{"query":"current weather in Seoul"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5bb146e3-b8de-4e13-b931-047b67d9cbc3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Seoul'}, 'id': 'call_HnDTh6kMR5HYZZzPRDf

## Streaming Messages

In [21]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the weather in Seoul?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the weather in Seoul?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_NOOhF1yvJdUDesotPIj6hbLp)
 Call ID: call_NOOhF1yvJdUDesotPIj6hbLp
  Args:
    query: current weather in Seoul
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in Seoul", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Seoul', 'region': '', 'country': 'South Korea', 'lat': 37.5664, 'lon': 126.9997, 'tz_id': 'Asia/Seoul', 'localtime_epoch': 1742281242, 'localtime': '2025-03-18 16:00'}, 'current': {'last_updated_epoch': 1742281200, 'last_updated': '2025-03-18 16:00', 'temp_c': 5.2, 'temp_f': 41.4, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 13.2, 'wind_

## Adding in memory

In [27]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

agent_executor.invoke({"messages": [HumanMessage(content="What is the weather in Seoul?")]}, config=config)

{'messages': [HumanMessage(content='What is the weather in Seoul?', additional_kwargs={}, response_metadata={}, id='1d59b220-5e66-4021-8be8-12275f990c21'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_e8xVYyySv7ghnIeUYVLC29c8', 'function': {'arguments': '{"query":"current weather in Seoul"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-307a3d5e-b42a-438e-be40-92b468b1379f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Seoul'}, 'id': 'call_e8xVY

In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the weather in Seoul?")]}, config=config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_max13UCq6ttTp22O4rKuijTs', 'function': {'arguments': '{"query":"current weather in Seoul"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 3432, 'total_tokens': 3454, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 3328}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d89980f8-4015-42c8-88e0-ba6a1e311e6b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Seoul'}, 'id': 'call_max13UCq6ttTp22O4rKuijTs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 3432, 'output_tokens': 22, 'total_tokens': 3454, 'i